In [11]:
#Nombre: ConteoPalabras
#Autor: PhD (C) Manuel Alejandro Pastrana Pardo
#Fecha de Creación: 2025-02-17
#Descripción: Clase que identifica el número de veces que aparece una palabra en una cadena de texto.
#Curso: Infraestructuras Paralelas y Distribuidas
#Código: 750023C

from collections import defaultdict
import threading

# Función para contar palabras en una sección del documento
def map_function(document_section, results, index):
    word_count = defaultdict(int)  # Usamos defaultdict para simplificar la suma
    for word in document_section.split():  # Dividimos por palabras completas
        word_count[word] += 1
    results[index] = word_count  # Guardamos el resultado en el índice correspondiente

# Función para combinar los resultados del mapeo
def reduce_function(mapped_results):
    total_count = defaultdict(int)
    for count in mapped_results:
        for word, num in count.items():
            total_count[word] += num
    return total_count

# Función principal para ejecutar el proceso MapReduce
def main(document):
    words = document.split()  # Dividimos el documento en una lista de palabras
    num_sections = 4  # Número de secciones a dividir
    section_size = len(words) // num_sections  # Tamaño de cada sección
    sections = [words[i * section_size:(i + 1) * section_size] for i in range(num_sections)]
    
    # Asegurarnos de incluir las palabras restantes (si no es divisible exactamente)
    if len(words) % num_sections != 0:
        sections[-1].extend(words[num_sections * section_size:])

    threads = []
    results = [None] * len(sections)  # Lista para almacenar los resultados del mapeo

    # Crear hilos para la fase de mapeo
    for i, section in enumerate(sections):
        thread = threading.Thread(target=map_function, args=(" ".join(section), results, i))
        threads.append(thread)
        thread.start()

    # Esperar a que todos los hilos terminen
    for thread in threads:
        thread.join()

    # Combinar resultados
    final_result = reduce_function(results)
    
    return final_result

# Ejemplo de uso
document = "palabra1 palabra2 palabra1 palabra3 palabra2 palabra1"
result = main(document)
print(result)

defaultdict(<class 'int'>, {'palabra1': 3, 'palabra2': 2, 'palabra3': 1})
